In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
load_dotenv(override = True)
api_key = os.getenv("OPENAI_API_KEY")

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()
    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"
    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [9]:
display_analysis("SBIN")

## Stock Analysis: State Bank of India (SBIN)

### Current Overview
- **Symbol:** SBIN.BO (BSE) / SBIN.NS (NSE)
- **Last Price:** ₹810.85 (BSE) / ₹810.95 (NSE)
- **Sector:** Financial Services
- **Type:** Stocks
- **Exchange:** BSE and NSE

### Understanding Stock Performance
The State Bank of India (SBIN) is a heavy player in the financial market with its shares currently priced around ₹810.85 to ₹810.95. This information provides a snapshot of the stock, but to assess its performance, we need to analyze the weekly price trends over the last year.

### Key Trends 
While specific historical weekly prices are not included in the data, here’s how to interpret the stock's trend based on general practices:

1. **Trends over Time:** 
   - **Going Upward:** If the stock has consistently closed at higher prices week over week, it indicates a bullish trend. Factors contributing to this could include strong quarterly earnings, economic growth prospects, or improvements in banking regulations.
   - **Volatility:** If the stock shows significant fluctuations (large highs and lows), it suggests instability. This may be due to macroeconomic factors like interest rate changes or lending policies.

2. **Closing Prices Analysis:**
   - If you observe the closing prices over the past weeks, you would look for patterns:
     - **Increasing Closing Prices:** Suggests investor confidence and might result from positive news or performance.
     - **Decreasing Closing Prices:** Could indicate worries regarding the bank's performance or industry issues.

### Total Returns Calculation
To calculate the expected returns percentage, we generally use the formula:

\[ \text{Total Return (\%)} = \left( \frac{\text{Ending Price} - \text{Beginning Price}}{\text{Beginning Price}} \right) \times 100 \]

Assuming the following hypothetical prices over a year:
- **Beginning Price:** ₹700 (a year ago)
- **Ending Price:** ₹810 (current)

Using the formula:

\[ \text{Total Return (\%)} = \left( \frac{810 - 700}{700} \right) \times 100 = \left( \frac{110}{700} \right) \times 100 \approx 15.71\% \]

### Summary
- SBIN is currently trading around ₹810.85, showing stability in the financial sector.
- A hypothetical return of approximately **15.71%** could have been expected if purchased a year ago at ₹700.
- It's advisable to look at detailed weekly price data for in-depth trends analysis to make informed investment decisions.

By understanding these metrics, new investors can gauge the health and trends of a stock like SBIN, ultimately helping them make better investment choices. Always consider various factors like market conditions and economic indicators before making investment decisions.

In [ ]:
display_analysis("ZOMATO.NS")

# Stock Analysis of ZOMATO.NS

### Overview
ZOMATO.NS is a stock listed on the National Stock Exchange in India. The provided data contains weekly trading information for approximately one year. For each week, we have the opening price, closing price, highest price, lowest price, closing price adjusted for splits/dividends, and trading volume.

### Key Terms
- **Open Price**: The stock price at the beginning of the trading week.
- **Close Price**: The stock price at the end of the trading week.
- **High Price**: The highest price achieved during the week.
- **Low Price**: The lowest price recorded during the week.
- **Adj Close**: The adjusted closing price that accounts for any stock splits and dividends.
- **Volume**: The total number of shares traded during the week.

### Trends Analysis
1. **Price Movement**:
   - The stock shows significant fluctuations over the year.
   - Initially, in January 2024, ZOMATO.NS was trading around ₹274.50. The prices then fluctuated, showing a general downtrend until June 2024 where it dipped down to around ₹179.15.
   - Starting from June 2024, there seems to be a steady rise in the stock price, reaching a high of ₹290.50 in September 2024 before experiencing another downtrend in the following months.

2. **Recent Performance**:
   - The most recent data point shows a closing price of ₹227.52 on March 21, 2025, recovering from a significantly low price before the February peak.
   - Overall, the stock witnessed some of its highest trading volumes during the period of October to December 2024, indicating higher investor interest.

3. **Volatility**:
   - The stock demonstrates a significant level of volatility, with weekly highs and lows indicating considerable price swings. For example, the highest recorded price was around ₹304.70 in December 2024.

### Total Return Calculation
To calculate the total return over the period, we can use the opening price from the first week and the closing price from the most recent week:

- **Opening Price (Jan 6, 2024)**: ₹274.50
- **Closing Price (Mar 21, 2025)**: ₹227.52

The formula for total return in percentage is:
\[
\text{Total Return (\%)} = \left( \frac{\text{Final Value} - \text{Initial Value}}{\text{Initial Value}} \right) \times 100
\]

Calculating the total return:
\[
\text{Total Return} = \left( \frac{227.52 - 274.50}{274.50} \right) \times 100 = \left( \frac{-46.98}{274.50} \right) \times 100 \approx -17.13\%
\]

### Summary
ZOMATO.NS has shown significant price volatility over the past year with notable peaks and valleys. Despite having periods of high prices, the overall return from January 2024 to March 2025 is approximately -17.13%. This indicates that the investment would have decreased in value. Investors should be cautious and consider market conditions before making any decisions on ZOMATO.NS.

In [10]:
# code to get the stock symbol and it's stock name from wiki page using BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_companies_listed_on_the_National_Stock_Exchange_of_India"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
    

In [12]:
title = soup.title.string if soup.title else "No title found"
for irrelevant in soup.body(["script", "style", "img", "input"]):
    irrelevant.decompose()

html_table_data = soup.find("table")


In [15]:
html_table_data.find_all("td")[1:10]  # Displaying first 10 rows of the table

[<td class="mbox-text"><div class="mbox-text-span">This article <b>needs additional citations for <a href="/wiki/Wikipedia:Verifiability" title="Wikipedia:Verifiability">verification</a></b>.<span class="hide-when-compact"> Please help <a href="/wiki/Special:EditPage/List_of_companies_listed_on_the_National_Stock_Exchange_of_India" title="Special:EditPage/List of companies listed on the National Stock Exchange of India">improve this article</a> by <a href="/wiki/Help:Referencing_for_beginners" title="Help:Referencing for beginners">adding citations to reliable sources</a>. Unsourced material may be challenged and removed.<br/><small><span class="plainlinks"><i>Find sources:</i> <a class="external text" href="https://www.google.com/search?as_eq=wikipedia&amp;q=%22List+of+companies+listed+on+the+National+Stock+Exchange+of+India%22" rel="nofollow">"List of companies listed on the National Stock Exchange of India"</a> – <a class="external text" href="https://www.google.com/search?tbm=nws&a

In [21]:
links = soup.find_all('table')

In [22]:
print(links)

[<table class="box-More_citations_needed plainlinks metadata ambox ambox-content ambox-Refimprove" role="presentation"><tbody><tr><td class="mbox-image"><div class="mbox-image-div"><span typeof="mw:File"><a class="mw-file-description" href="/wiki/File:Question_book-new.svg"></a></span></div></td><td class="mbox-text"><div class="mbox-text-span">This article <b>needs additional citations for <a href="/wiki/Wikipedia:Verifiability" title="Wikipedia:Verifiability">verification</a></b>.<span class="hide-when-compact"> Please help <a href="/wiki/Special:EditPage/List_of_companies_listed_on_the_National_Stock_Exchange_of_India" title="Special:EditPage/List of companies listed on the National Stock Exchange of India">improve this article</a> by <a href="/wiki/Help:Referencing_for_beginners" title="Help:Referencing for beginners">adding citations to reliable sources</a>. Unsourced material may be challenged and removed.<br/><small><span class="plainlinks"><i>Find sources:</i> <a class="externa

In [23]:
# Find all rows
for row in soup.find_all('tr'):
    tds = row.find_all('td')
    if tds:
        # Get all <a> tags inside the first <td>
        a_tags = tds[0].find_all('a', class_='external text')
        if a_tags:
            symbol = a_tags[0].text.strip()
            print(symbol)

20MICRONS
21STCENMGM
3IINFOTECH
3MINDIA
3PLAND
3RDROCK
5PAISA
63MOONS
AMJUMBO
ABINFRA
ABNINT
A2ZINFRA
AAKASH
AARON
AARTIDRUGS
AARTIIND
AARVEEDEN
AARVI
AAVAS
ABAN
ABB
POWERINDIA
ABMINTLTD
ACC
ACCELYA
ACCORD
ACCURACY
ACEINTEG
ACE
ADANIENT
ADANIGAS
ADANIGREEN
ADANIPORTS
ADANIPOWER
ADANITRANS
ADFFOODS
ADHUNIKIND
ABCAPITAL
ABFRL
BIRLAMONEY
ADLABS
ADORWELD
ADROITINFO
ADVENZYMES
ADVANIHOTR
AEGISCHEM
AFFLE
AGARIND
AGCNET
AGRITECH
AGROPHOS
ATFL
AHIMSA
AHLADA
AHLUCONT
AIAENG
AIRAN
AIROLAM
AJANTPHARM
AJMERA
AJOONI
AKASH
AKG
AKSHOPTFBR
AKSHARCHEM
AKZOINDIA
ALANKIT
ALBERTDAVD
ALCHEM
ALEMBICLTD
APLLTD
ALICON
ALKALI
ALKEM
ALKYLAMINE
ALLCARGO
ADSL
ALLSEC
ALMONDZ
ALOKINDS
ALPA
ALPHAGEO
ALPSINDUS
AMARAJABAT
AMBANIORG
AMBER
AMBIKCO
AMBUJACEM
AMDIND
ASIL
AMJLAND
AMRUTANJAN
ANANTRAJ
ANDHRACEMT
ANDHRAPAP
ANGELBRKG
AISL
ANIKINDS
APCL
ANKITMETAL
ANSALHSG
ANSALAPI
APARINDS
APCOTEXIND
APEX
APLAPOLLO
APOLLOHOSP
APOLLO
APOLLOPIPE
APOLSINHOT
APOLLOTYRE
APTECHT
ARCHIDPLY
ARCHIES
ARCOTECH
ARIES
ARIHANT
ARIHANTSUP
AR

In [1]:
import pandas as pd